# Assess Task Compute Time and Data Sizes
To get a better idea how they are each different and provide quantitative descriptions in the paper

In [1]:
from pathlib import Path
import pandas as pd
import json

Configuration

In [2]:
run_dir = Path('prod-runs/baseline_retrain-8/')

## Load in Each Type of Task
We'll need the runtimes and method sizes

In [3]:
tasks = []
for path in run_dir.glob('*-results.json'):
    records = pd.read_json(path, lines=True)
    records['total_size'] = records['message_sizes'].apply(lambda x: sum(x.values())) / 1e6
    tasks.append(records[['method', 'time_running', 'total_size', 'message_sizes']])
tasks = pd.concat(tasks)

## Group by Mean
Print out the statistics

In [4]:
tasks.groupby('method').describe(percentiles=[0.5])

time_running                                                  \
                      count        mean         std         min         50%   
method                                                                        
evaluate               24.0    1.299441    0.115573    1.167516    1.276802   
run_calculator        212.0  364.078799  300.695078    7.020325  315.904662   
run_sampling          146.0   10.024775    4.893819    1.054254   11.555648   
train                  88.0  262.904165    6.502937  251.004341  261.928531   

                            total_size                                  \
                        max      count       mean       std        min   
method                                                                   
evaluate           1.548957       24.0   9.920089  0.130585   9.751855   
run_calculator  1077.205258      215.0   0.019430  0.007756   0.005884   
run_sampling      17.632953      146.0   3.290176  0.119985   3.103596   
train            275.500636       88.0  20.947684  0.611232  19.986104   

                                      
                      50%        max  
method                                
evaluate         9.946885  10.061527  
run_calculator   0.021130   0.031874  
run_sampling     3.301844   3.552560  
train           20.934614  21.918548

## Get the size of the inference task
How many structures per chunk

In [5]:
with (run_dir / 'runparams.json').open() as fp:
    config = json.load(fp)

In [6]:
config['infer_chunk_size']

500